# Day 05: If You Give A Seed A Fertilizer

## Setup

In [1]:
import sys

data = [line for line in open("./inputs/day05.txt", "r").read().splitlines()]

## Part 1
First, get all the seeds (starting number), then iterate through all the maps and save the lowest mapped number. Name of the map is saved to ensure that map can't occur more than once.

In [2]:
seeds = [int(seed) for seed in data[0].split(': ')[1].split(" ")]

min_result = sys.maxsize

for seed in seeds:
    map_name = ""
    performed_maps = set()

    for i in range(2, len(data)):
        if len(data[i]) == 0:
            continue

        if not data[i][0].isnumeric():
            map_name = data[i]
            continue

        if map_name in performed_maps:
            continue

        destination, source, range_length = [int(n) for n in data[i].split(" ")]

        if seed in range(source, source + range_length):
            performed_maps.add(map_name)
            seed += (destination - source)

    if seed < min_result:
        min_result = seed

print(min_result)

177942185


## Part 2
Let's flip this shit around

In [ ]:
seed_ranges = []
mappings = []

map_name = ""
map_index = -1
map_ranges = []
for i in range(len(data)):
    if i == 0:
        seed_input = [int(seed) for seed in data[0].split(': ')[1].split(" ")]
        for s, seed in enumerate(seed_input):
            if s % 2 == 0:
                seed_ranges.append(range(seed, seed + seed_input[s + 1]))

        continue

    if len(data[i]) == 0:
        continue

    if not data[i][0].isnumeric():
        map_name = data[i]
        map_index += 1

        if map_index < 1:
            continue

        mappings.insert(map_index, map_ranges)
        map_ranges = []

        continue

    destination, source, range_length = [int(n) for n in data[i].split(" ")]
    map_ranges.append([map_name, set(range(source, source + range_length)), destination - source])

mappings.insert(map_index, map_ranges)
print(mappings)
min_result = sys.maxsize
# 
# for seed_range in seed_ranges:
#     queue = [[seed_range]]
# 
#     for map_id, mapping in enumerate(mappings):
#         while len(queue[map_id]) > 0:
# 
#             seed = queue[map_id].pop()
# 
#             next_map_id = map_id + 1
#             for mapping_name, mapping_range, offset in mapping:
#                 intersection = mapping_range.intersection(seed)
#                 if intersection:
#                     # print("queuing", next_map_id, range(min(intersection) + offset, max(intersection) + offset), "from", seed, "with offset", offset, mapping_name)
# 
#                     if len(queue) <= next_map_id:
#                         queue.insert(next_map_id, [range(min(intersection) + offset, max(intersection) + offset)])
#                     else:
#                         queue[next_map_id].append(range(min(intersection) + offset, max(intersection) + offset))
# 
#             if len(queue) <= next_map_id:
#                 queue.insert(next_map_id, [seed])
# 
#     min_seed = min([seed.start for seed in queue[7]])
#     if min_seed < min_result:
#         min_result = min_seed

print(min_result)